In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [6]:
train_data_dir = 'Dataset/Train'

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [8]:
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

Found 1216 images belonging to 4 classes.


In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

# Load InceptionV3 model
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom fully connected layers
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model
model = tf.keras.models.Model(base_model.input, x)

# Add EarlyStopping callback to monitor training loss
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Compile the model with the callback
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['acc'])

# Train the model with early stopping using generator
inception_hist = model.fit(train_generator, 
                           steps_per_epoch=20, 
                           epochs=10, 
                           callbacks=[early_stopping])

# Print the final accuracy
print("Final Accuracy: ", inception_hist.history['acc'][-1])


2024-03-04 15:43:46.743106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 15:43:46.780890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 15:43:46.781235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 15:43:46.781965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10


2024-03-04 15:43:54.691372: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-03-04 15:43:56.062370: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-04 15:43:56.064535: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-04 15:43:56.064585: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-03-04 15:43:56.066591: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-04 15:43:56.066644: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-03-04 15:43:57.107206: I tensorflow/stream_executor/cuda/c

20/20 [==============================] - 12s 267ms/step - loss: 0.6984 - acc: 0.6812
Epoch 2/10
20/20 [==============================] - 5s 264ms/step - loss: 0.6413 - acc: 0.7045
Epoch 3/10
20/20 [==============================] - 5s 264ms/step - loss: 0.6028 - acc: 0.7311
Epoch 4/10
20/20 [==============================] - 5s 242ms/step - loss: 0.6041 - acc: 0.7300
Epoch 5/10
20/20 [==============================] - 5s 246ms/step - loss: 0.5856 - acc: 0.7450
Epoch 6/10
20/20 [==============================] - 5s 247ms/step - loss: 0.5793 - acc: 0.7462
Epoch 7/10
20/20 [==============================] - 5s 254ms/step - loss: 0.5754 - acc: 0.7500
Epoch 8/10
20/20 [==============================] - 5s 244ms/step - loss: 0.5726 - acc: 0.7500
Epoch 9/10
20/20 [==============================] - 5s 253ms/step - loss: 0.5701 - acc: 0.7500
Epoch 10/10
20/20 [==============================] - 6s 298ms/step - loss: 0.5687 - acc: 0.7500
Final Accuracy:  0.75


https://www.kaggle.com/datasets/matshidiso/soil-types